### Projeto analise de series temporais utilizando dados de vendas

Objetivo: prever vendas utilizando dados de vendas ocorridas no período de 2016-06-01 to 2022-01-01

##IMPORTACAO DOS DADOS
    Importar a tabela

##TRATAMENTO DOS DADOS

##ANALISE DOS DADOS
    
       
##CONSTRUCAO DOS GRAFICOS DE SERIES TEMPORAIS: 
    Tendencia, sazonalidade, etc e modelo.  

In [1]:
import numpy as np
#from matplotlib import pylot as plt
import pandas as pd

from pmdarima import auto_arima                  
from statsmodels.tsa.arima.model import ARIMA

from xgboost import XGBRegressor

# Importar a tabela de dados


df = pd.read_excel(r"MachineOrders03.xlsx")
df.head(5)
#display(df)

,Custno,Invdate,year,yearmonth,Invoice,SONO,Partno,descript,prodcl,Product Class,soldby,txnprice,Cur,Rate,totprice
0,294,2016-06-26,2016,2016-06,80956,70993.0,JSE8500,MACHINE GLAZE SPRAYER ECONOMY,72,Jelly Sprayer,HA,0.0000,CAN,1.000000,3890.00
1,2584,2016-06-27,2016,2016-06,80972,70435.0,COM1000iSV-EU-B,CAKEOMATIC 1000iSV - DEPOSIT EU,17,Cake-o-Matic,RO,0.0000,CAN,1.000000,21628.50
2,2584,2016-06-27,2016,2016-06,80972,70435.0,UVL1000iSV,UNIVERSAL 1000iSV,10,Universal,RO,0.0000,CAN,1.000000,27338.37
3,1258,2016-06-28,2016,2016-06,80975,70457.0,CDT3010SV-EU,COMPACT TABLETOP - SV EU NO-TABLE,34,Compact,MH,4486.7846,EUR,0.729927,6146.90
4,1258,2016-06-28,2016,2016-06,80975,70457.0,CDT3010SV-EU,COMPACT TABLETOP - SV EU NO-TABLE,34,Compact,MH,4486.7846,EUR,0.729927,6146.90


In [2]:
# Deletar as colunas q nao necessarias

new_tab = df.drop(["Custno", "yearmonth", "year", "Invoice", "SONO", "Partno","descript", "prodcl", "Product Class", "soldby","txnprice", "Cur", "Rate"], axis = 1) 
display(new_tab)

,Invdate,totprice
0,2016-06-26,3890.00
1,2016-06-27,21628.50
2,2016-06-27,27338.37
3,2016-06-28,6146.90
4,2016-06-28,6146.90
...,...,...
6197,2022-01-20,35206.27
6198,2022-01-20,69353.79
6199,2022-01-20,74265.18
6200,2022-01-20,13780.80


In [3]:
new_tab.index.min(), new_tab.index.max()

(0, 6201)

In [4]:
new_tab.info()

#df.dtypes
#df.columns  
#list(df.columns) #print(df.isna().sum()) -> mostra qtde celulas vazias

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6202 entries, 0 to 6201
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Invdate   6202 non-null   datetime64[ns]
 1   totprice  6202 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 97.0 KB


In [7]:
pd.to_datetime(new_tab.Invdate)

0      2016-06-26
1      2016-06-27
2      2016-06-27
3      2016-06-28
4      2016-06-28
          ...    
6197   2022-01-20
6198   2022-01-20
6199   2022-01-20
6200   2022-01-20
6201   2022-01-20
Name: Invdate, Length: 6202, dtype: datetime64[ns]

In [ ]:
#pd.to_datetime(df.yearmonth) # = object -> transf. eh a conversao da coluna yearmonth p/datetime usando df.set
#nessa transf. ele definiu tdo como dia 1 ja q nao tem dia
#nesse codigo acima, o dataframe n foi alterado. Entao o codigo abaxio jogara essa alteracao na mesma coluna

#df.yearmonth = pd.to_datetime(df.yearmonth) 
#df.info()

In [ ]:
#agora transf essa nova coluna yearmonth para ser o indice 
#df.set_index("yearmonth", inplace=True)


In [ ]:
#df.info() # com essa transf. q antes era RangeIndex agora se tornou DatetimeIndex. Periodo:2016-06-01 a 2022-01-01


In [8]:
round(new_tab.describe())

,totprice
count,6202.0
mean,22644.0
std,32288.0
min,2004.0
25%,8538.0
50%,12979.0
75%,23513.0
max,730000.0


In [ ]:
#excluir outlier

In [16]:
#new_tab["totprice"].plot(figsize=(10,6))



###Criando a variavel target (variavel criada manualmente que eh o valor q quero prever - puxei valor da linha 2 para 1, a linha 3 p/ 2 e assim sucessivamente)

In [10]:
#total price do historico - ontem
new_tab["totprice"]

0         3890.00
1        21628.50
2        27338.37
3         6146.90
4         6146.90
          ...    
6197     35206.27
6198     69353.79
6199     74265.18
6200     13780.80
6201    115568.65
Name: totprice, Length: 6202, dtype: float64

In [11]:
#total price de amanha (criacao de uma variavel defasando ela em um dia. Isso permitira prever um dia posterior )

new_tab["totprice"].shift(-1)

0        21628.50
1        27338.37
2         6146.90
3         6146.90
4        23588.34
          ...    
6197     69353.79
6198     74265.18
6199     13780.80
6200    115568.65
6201          NaN
Name: totprice, Length: 6202, dtype: float64

In [12]:
#comparar os valores da coluna preco "deslocado" (desejado) com o historico (totprice)
new_tab["desejado"] = new_tab["totprice"].shift(-1)
new_tab.head()

,Invdate,totprice,desejado
0,2016-06-26,3890.00,21628.50
1,2016-06-27,21628.50,27338.37
2,2016-06-27,27338.37,6146.90
3,2016-06-28,6146.90,6146.90
4,2016-06-28,6146.90,23588.34


In [13]:
#tabela mostrando que por ter puxado os valores da coluna, a ultima celula ficara sem informacao (subiu uma)
#o ultimo dia nao havera o "amanha" que sera o dia que quero prever
new_tab.tail()

,Invdate,totprice,desejado
6197,2022-01-20,35206.27,69353.79
6198,2022-01-20,69353.79,74265.18
6199,2022-01-20,74265.18,13780.80
6200,2022-01-20,13780.80,115568.65
6201,2022-01-20,115568.65,NaN


In [14]:
#Por ter puxado os valores (passo acima), a ultima celula "NaN" precisa ser deletada

new_tab = new_tab.dropna()
new_tab.tail()

,Invdate,totprice,desejado
6196,2022-01-20,21083.65,35206.27
6197,2022-01-20,35206.27,69353.79
6198,2022-01-20,69353.79,74265.18
6199,2022-01-20,74265.18,13780.80
6200,2022-01-20,13780.80,115568.65
